# Prática

Agora que vimos a teoria, vamos implementar o algoritmo de Grover usando ket.
Aqui, cada estado da base computacional será representado na base decimal, $\left| i \right\rangle, i = 0, \dots, 2^n - 1$.
Logo, precisamos passar um inteiro maior ou igual a 0 que indicará qual desses estados queremos marcar, também será necessário especificarmos quantos bits serão simulados.
Primeiro vamos importar o necessário.

In [1]:
from math import sqrt, pi
from ket import *


Para facilitar, vamos dividir o algoritmo em quatro partes:

1. Oráculo de fase;
2. Difusor;
3. Operador de Grover; e a
4. Função principal ``grover()``, a qual será chamada e realizará todas as operações necessárias.

## Oráculo de fase

In [2]:
def phase_oracle(qubits: quant, state: int):
    with around(flipc(state), qubits):
        ctrl(qubits[:-1], Z, qubits[-1])


O que estamos fazendo nesse código é receber os qubits e o estado desejado como argumentos e aplicando a porta $Z$ controlada. Se os qubits estiverem no estado `state` aplicaremos a porta `Z` no último qubit (``qubit[-1]``). Como essa operação controlada aplicada a fase -1 apenas quando os qubits estão no estado $\left|1\right>$, usamos a porta `flipc` para parmutar o estdo desejado (`state`) para o estado de controle ($\left|1\cdots1\right>$). Como o `with around` garatimos que a permitação é desfeita apos a aplicação da porta controlada.

## Difusor

O difusor é muito semelhante ao código do oráculo, porém nele iremos aplicar a porta de hadamard antes e depois da porta controlada, e o estado a ser marcado é $\left| 0 \right\rangle$.

In [3]:
def grover_diffuser(qubits: quant):
    with around(H, qubits):
        phase_oracle(qubits, 0)


## Operador de Grover

Agora partindo para o operador de Grover, temos o código mais simples, consistindo apenas da chamada do oráculo e depois do difusor.


In [4]:
def grover_operator(qubits: quant, state: int):
    phase_oracle(qubits, state)
    grover_diffuser(qubits)


## Função principal

Para finalizar, temos a função principal, a qual irá criar os qubits e definir quantas vezes precisaremos aplicar o operador de Grover.


In [5]:
def grover(state: int, num_qubits: int) -> int:
    qubits = quant(num_qubits)
    entries = 2**num_qubits
    steps = int((pi / 4) * sqrt(entries))

    H(qubits)

    for _ in range(steps):
        grover_operator(qubits, state)

    return measure(qubits).value


Após todas operações medimos e retornamos o valor obtido.
Como se trata de um algoritmo probabilístico, nem sempre teremos o estado desejado como resultado, porém é esperado que tenha alta taxa de sucesso.
Com tudo pronto podemos chamar a função ``grover()``.

In [6]:
from random import randrange
num_qubits = 10
for state in [randrange(2**num_qubits) for _ in range(10)]:
    print('Procurando estado', state, '...', end=' ')
    print('Estado medido:', grover(state, num_qubits))


Procurando estado 788 ... Estado medido: 788
Procurando estado 794 ... Estado medido: 794
Procurando estado 593 ... Estado medido: 593
Procurando estado 410 ... Estado medido: 410
Procurando estado 594 ... Estado medido: 594
Procurando estado 897 ... Estado medido: 897
Procurando estado 516 ... Estado medido: 516
Procurando estado 622 ... Estado medido: 622
Procurando estado 822 ... Estado medido: 822
Procurando estado 802 ... Estado medido: 802
